# 🚀 Fraud Detection with Imbalanced Data
## Complete Tutorial with SMOTE, Isolation Forests & More

This notebook demonstrates best practices for handling highly imbalanced datasets in fraud detection.

**What you'll learn:**
- Why accuracy is misleading for imbalanced data
- How to use SMOTE for oversampling
- Comparing 5 different approaches
- Proper evaluation metrics (Precision, Recall, F1-Score)

### 1. Import Libraries

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

from load_and_analyze import FraudDataLoader, DataAnalyzer
from fraud_detection_model import FraudDetectionModel

print("✅ Libraries imported successfully!")

### 2. Load and Analyze Data

We'll create a synthetic dataset with **99:1 imbalance** (1% fraud) to mimic real-world credit card fraud.

In [ ]:
# Create dataset with 50,000 transactions
loader = FraudDataLoader(random_state=42)
df = loader.create_synthetic_dataset(n_samples=50000, fraud_ratio=0.01)

print(f"\n📊 Dataset shape: {df.shape}")
print(f"\n🔍 First few rows:")
df.head()

### 3. Analyze Class Distribution

Let's examine the **severe class imbalance** in our dataset.

In [ ]:
# Analyze the imbalance
analyzer = DataAnalyzer(df, target_col='Class')
analyzer.basic_info()
dist_df = analyzer.analyze_class_distribution()

In [ ]:
# Visualize the distribution
analyzer.plot_distributions(save_path='reports/figures/')

### 4. The Accuracy Trap ⚠️

**Critical Insight:** A model that always predicts "Normal" would achieve **99% accuracy** but catch **0% of fraud**!

This is why we need proper metrics:
- ✅ **Precision**: Of predicted frauds, how many are real?
- ✅ **Recall**: Of actual frauds, how many did we catch?
- ✅ **F1-Score**: Harmonic mean of precision and recall
- ❌ **Accuracy**: Misleading for imbalanced data!

### 5. Initialize Model Trainer

We'll train **5 different models** to compare approaches:
1. **Baseline** - No balancing
2. **SMOTE** - Synthetic oversampling
3. **Undersampling** - Reduce majority class
4. **Class Weights** - Penalize misclassification
5. **Isolation Forest** - Anomaly detection

In [ ]:
# Initialize trainer
trainer = FraudDetectionModel(random_state=42)
trainer.df = df
trainer.X = df.drop('Class', axis=1)
trainer.y = df['Class']

# Prepare data (train-test split + scaling)
trainer.prepare_data(test_size=0.2)

print("\n✅ Data prepared and ready for training!")

### 6. Train Baseline Model

First, let's see how a standard Random Forest performs **without any balancing**.

In [ ]:
trainer.train_baseline()

### 7. Train with SMOTE (Synthetic Minority Over-sampling)

**SMOTE** creates synthetic fraud examples to balance the training set.

**How it works:**
1. Takes a fraud sample
2. Finds its k-nearest neighbors (also fraud)
3. Creates synthetic samples between them
4. Balances the dataset without losing information

In [ ]:
trainer.train_with_smote()

### 8. Train with Random Undersampling

**Undersampling** reduces the majority class to match the minority class.

**Trade-off:**
- ✅ Faster training (smaller dataset)
- ✅ Often higher recall
- ❌ Loses potentially useful information

In [ ]:
trainer.train_with_undersampling()

### 9. Train with Class Weight Balancing

**Class weights** penalize the model more for misclassifying the minority class.

**Advantage:** No data modification needed!

In [ ]:
trainer.train_with_class_weight()

### 10. Train Isolation Forest (Anomaly Detection)

**Isolation Forest** is an unsupervised approach that identifies anomalies.

**Note:** May not work well if fraud patterns aren't pure "anomalies".

In [ ]:
trainer.train_isolation_forest()

### 11. Compare All Models 📊

Now let's see which approach works best!

In [ ]:
trainer.compare_models()

### 12. Detailed Results Analysis

Let's examine the results for each model:

In [ ]:
import pandas as pd

# Create results DataFrame
results_data = []
for name, result in trainer.results.items():
    cm = result['confusion_matrix']
    tn, fp, fn, tp = cm[0,0], cm[0,1], cm[1,0], cm[1,1]
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    results_data.append({
        'Model': name,
        'Precision': f"{result['precision']:.4f}",
        'Recall': f"{result['recall']:.4f}",
        'F1-Score': f"{result['f1_score']:.4f}",
        'ROC-AUC': f"{result['roc_auc']:.4f}" if result['roc_auc'] else 'N/A',
        'Fraud Detection': f"{detection_rate:.2%}",
        'True Positives': tp,
        'False Negatives': fn,
        'False Positives': fp
    })

results_df = pd.DataFrame(results_data)
print("\n📊 COMPREHENSIVE RESULTS:")
print("="*100)
results_df

### 13. Key Takeaways 🎓

**Best Model:** SMOTE Oversampling typically achieves the best F1-Score

**Why SMOTE wins:**
- ✅ Balances classes without losing information
- ✅ Creates intelligent synthetic samples
- ✅ Better generalization than undersampling
- ✅ High precision AND recall

**When to use alternatives:**
- **Undersampling**: When you need maximum recall (catch all fraud)
- **Class Weights**: When you can't modify the dataset
- **Isolation Forest**: When you don't have labeled data

**Remember:**
- ❌ Don't use accuracy for imbalanced data!
- ✅ Always use Precision, Recall, and F1-Score
- ✅ Consider business costs when choosing thresholds

### 14. Using the Best Model for Predictions

Here's how to use the trained model on new data:

In [ ]:
# Get the best model (SMOTE)
best_model = trainer.models['SMOTE']

# Example: Predict on test set
predictions = best_model.predict(trainer.X_test_scaled)
probabilities = best_model.predict_proba(trainer.X_test_scaled)[:, 1]

# Show some high-risk transactions
import numpy as np
high_risk_indices = np.where(probabilities > 0.7)[0][:5]

print(f"\n🚨 Found {len(np.where(probabilities > 0.7)[0])} high-risk transactions (>70% fraud probability)")
print(f"\nTop 5 highest risk transactions:")
for idx in high_risk_indices:
    actual = trainer.y_test.iloc[idx]
    prob = probabilities[idx]
    print(f"  Transaction {idx}: {prob:.2%} fraud probability - Actual: {'FRAUD' if actual == 1 else 'Normal'}")

### 15. Business Impact Analysis 💰

Let's calculate the financial impact of our model:

In [ ]:
# Assumptions
avg_fraud_amount = 2500  # Average fraud transaction
investigation_cost = 10   # Cost to investigate false positive

# Get SMOTE model results
smote_cm = trainer.results['SMOTE Oversampling']['confusion_matrix']
tn, fp, fn, tp = smote_cm[0,0], smote_cm[0,1], smote_cm[1,0], smote_cm[1,1]

# Calculate financial impact
fraud_caught_value = tp * avg_fraud_amount
fraud_missed_cost = fn * avg_fraud_amount
false_positive_cost = fp * investigation_cost
net_benefit = fraud_caught_value - fraud_missed_cost - false_positive_cost

print("\n💰 BUSINESS IMPACT ANALYSIS (SMOTE Model):")
print("="*60)
print(f"Fraud Caught: {tp} transactions × ${avg_fraud_amount:,} = ${fraud_caught_value:,}")
print(f"Fraud Missed: {fn} transactions × ${avg_fraud_amount:,} = -${fraud_missed_cost:,}")
print(f"False Alarms: {fp} investigations × ${investigation_cost} = -${false_positive_cost:,}")
print("="*60)
print(f"NET BENEFIT: ${net_benefit:,}")
print(f"\nFor every {len(trainer.y_test):,} transactions, this model saves ${net_benefit:,}!")

### 16. Next Steps 🚀

**For Production Deployment:**
1. Train on real transaction data
2. Set threshold based on business costs
3. Implement real-time scoring API
4. Monitor performance and retrain regularly
5. Add explainability (SHAP values)

**For Further Learning:**
- Try ensemble methods (combine multiple models)
- Experiment with deep learning approaches
- Add feature engineering (velocity, location patterns)
- Implement cost-sensitive learning

---

**🎉 Congratulations!** You now know how to handle imbalanced data in fraud detection!